## 수치 미분을 이용한 심층 신경망 학습

In [2]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## 유틸리티 함수

In [3]:
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(h, y):
    return 1 / 2 * np.mean(np.square(h - y))

## 뉴런 구현

In [5]:
class Neuron:
    def __init__(self, W, b, a):
        # Model Parameter
        self.W = W
        self.a = a
        self.b = b
        
        # Gradients
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)

    def __call__(self, x):
        return self.a(_m(_t(self.W),x) + self.b) # activation((W^T)x + b)

## 심층신경망 구현

In [14]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))

        self.sequence = list()
        # First hidden layer
        W, b = init_var(num_input, num_neuron)
        self.sequence.append(Neuron(W, b, activation))
        
        # Hidden layers
        for _ in range(hidden_depth - 1):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W, b, activation))

        # Output layer
        W, b = init_var(num_neuron, num_output)
        self.sequence.append(Neuron(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, x, y, loss_func):
        def get_new_sequence(layer_index, new_neuron):
            new_sequence = list()
            for i, layer in enumerate(self.sequence):
                if i == layer_index:
                    new_sequence.append(new_neuron)
                else:
                    new_sequence.append(layer)
            return new_sequence
        
        def evel_sequence(x, sequence):
            for layer in sequence:
                x = layer(x)
            return x
        
        loss = loss_func(self(x), y)
        
        for layer_id, layer in enumerate(self.sequence): # iterate layer
            for w_i, w in enumerate(layer.W): # iterate W (row)
                for w_j, ww in enumerate(w): # iterate W (col)
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon
                    
                    new_neuron =  Neuron(W, layer.b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = evel_sequence(x, new_seq)
                    
                    num_grad = (loss_func(h, y) - loss) / epsilon # (f(x + eps) - f(x)) / epsilon
                    layer.dW[w_i][w_j] = num_grad
                    
                for b_i, bb in enumerate(layer.b): # iterate b
                    b = np.copy(layer.b)
                    b[b_i] = bb + epsilon
                    
                    new_neuron =  Neuron(layer.W, b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = evel_sequence(x, new_seq)
                    
                    num_grad = (loss_func(h, y) - loss) / epsilon # (f(x + eps) - f(x)) / epsilon
                    layer.db[b_i] = num_grad
        return loss

## 경사하강 학습법

In [15]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = network.calc_gradient(x, y, loss_obj)
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

## 동작 테스트

In [16]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.3664178220839986
Epoch 1: Test loss 0.36436275113433253
Epoch 2: Test loss 0.3623197940618879
Epoch 3: Test loss 0.3602892259066692
Epoch 4: Test loss 0.35827131057299966
Epoch 5: Test loss 0.35626630076048116
Epoch 6: Test loss 0.3542744379166652
Epoch 7: Test loss 0.3522959522107293
Epoch 8: Test loss 0.3503310625282303
Epoch 9: Test loss 0.3483799764853359
Epoch 10: Test loss 0.3464428904626653
Epoch 11: Test loss 0.34451998965770636
Epoch 12: Test loss 0.34261144815505234
Epoch 13: Test loss 0.3407174290135133
Epoch 14: Test loss 0.3388380843697079
Epoch 15: Test loss 0.33697355555689323
Epoch 16: Test loss 0.3351239732383792
Epoch 17: Test loss 0.3332894575545421
Epoch 18: Test loss 0.3314701182825104
Epoch 19: Test loss 0.32966605500808427
Epoch 20: Test loss 0.3278773573081994
Epoch 21: Test loss 0.32610410494376557
Epoch 22: Test loss 0.3243463680616629
Epoch 23: Test loss 0.3226042074050963
Epoch 24: Test loss 0.320877674531523
Epoch 25: Test loss 0.319166